In [15]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb, m, p, H, cat
from DGM import DGMPIASolver, DeepPDESolver
from pdes import HBJ, PDE
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG
from FBSDEs import FBSDESolver
from pdes import FBSDE
from sampling import PATH_SPACES
import os
import torch.distributions as D

In [16]:
rates = 1
exp = torch.poisson(torch.ones(10, 5))
exp

tensor([[1., 2., 1., 1., 3.],
        [3., 2., 1., 0., 0.],
        [3., 1., 0., 1., 1.],
        [1., 2., 2., 4., 3.],
        [2., 0., 0., 1., 1.],
        [1., 0., 0., 0., 1.],
        [0., 0., 2., 1., 1.],
        [2., 2., 0., 1., 0.],
        [4., 1., 3., 2., 0.],
        [1., 1., 0., 1., 1.]])

In [17]:
batch_size = 32
var_dim = 100
def sampling(var, domain=(-1, 1)):
    rates = 0.4
    exp = D.exponential.Exponential(rates)
    #means_clusters = torch.clamp(torch.round(exp.sample((var[-1].shape[0], var_dim))), min=0, max=var_dim-1).long()
    means_clusters = torch.randint(1, var_dim, size=(var[-1].shape[0], var_dim))
    bias = ((torch.rand((var[-1].shape[0], 1)))-0.5)*0.4
    means = (torch.rand((var[-1].shape[0], var_dim))-0.5)*1.6 + bias
    samples = []
    for i in range(len(var)):
        m = means.gather(-1, means_clusters[:, None, i])
        b = torch.min(torch.abs(m-domain[0]), torch.abs(m-domain[1]))
        c = torch.rand((var[-1].shape[0], 1))
        samples.append((var[i]-0.5)*torch.min(c, b) + m)
        # need gap between
    return samples

In [18]:
samples = [torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(var_dim)]
samples = torch.cat(sampling(samples), dim=-1).numpy()

In [19]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [20]:
samples = [(torch.zeros(size=(batch_size, 1)).uniform_()-0.5)*2 for _ in range(var_dim)]
samples = torch.cat(samples, dim=-1).numpy()

In [21]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [22]:
mix = D.Categorical(torch.rand(5,))
comp = D.Independent(D.Normal(
            torch.randn(5, var_dim), torch.rand(5, var_dim)), 1)
gmm = D.MixtureSameFamily(mix, comp)

In [23]:
def sampm():
    mix = D.Categorical(torch.rand(10,))
    comp = D.Independent(D.Normal(torch.randn(10, var_dim)*0.1 + torch.rand(10, var_dim)-0.5, torch.rand(10, var_dim)), 1)
    gmm = D.MixtureSameFamily(mix, comp)
    return gmm.sample(sample_shape=torch.Size([batch_size]))

In [24]:
samples = sampm().numpy()

In [25]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()

In [29]:
batch_size = 32
var_dim = 10
def sampling(var, domain=(-1, 1)):
    exp = D.Gamma(1, 2)
    means_clusters = torch.clamp(torch.round(exp.sample((var[-1].shape[0], var_dim))), min=0, max=var_dim-1).long()
    #means_clusters = torch.randint(1, var_dim, size=(var[-1].shape[0], var_dim))
    means = (torch.rand((var[-1].shape[0], var_dim))-0.5)*2
    samples = []
    for i in range(len(var)):
        m = means.gather(-1, means_clusters[:, None, i])
        b = torch.min(torch.abs(m-domain[0]), torch.abs(m-domain[1]))
        c = torch.rand((var[-1].shape[0], 1))
        samples.append((var[i]-0.5)*torch.min(c, b) + m)
        # need gap between
    return samples

In [30]:
samples = [torch.zeros(size=(batch_size, 1)).uniform_() for _ in range(var_dim)]
samples = torch.cat(sampling(samples), dim=-1).numpy()

In [31]:
fig = make_subplots(rows=1, cols=1)
for i in range(samples.shape[0]):
    fig.add_trace(go.Scatter(x=samples[i].flatten(), y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='SkyBlue', size=10), showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[np.mean(samples[i].flatten())], y=np.ones(var_dim)*i,
                         mode='markers', marker=dict(color='red', size=10), showlegend=False), row=1, col=1)
fig.update_layout(
    title="Opinion distribution",
    xaxis_title="X",
    yaxis_title="Num",
    font=dict(
        family="Courier New, monospace",
        size=14
    )
)
fig.show()